This notebook is to confirm the accuracy of VHS 1256 b's coordinate.

P1386 (ERS, same as Gauza+2015):     
RA: 12h56m01s.586 Dec: −12°57′30.′′310 Epoch: MJD: 55743.067635     
pmra: -275.4 mas/yr pmdec: -198.4 mas/yr    

Current APT (APT, probably Gaia):    
RA: 12 56 1.5389 Dec: -12 57 30.62 Epoch: 2015.5    
pmra: -0.018949 s/yr = -284.2 mas/yr pmdec = -189.0 mas/yr    

SIMBAD host (A_simbad, gaia):     
RA: 12 56 02.1337046541 Dec: -12 57 21.924307013 Epoch: J2000.0     
pmra: -272.5 mas/yr pmdec = -190.2 mas/yr   

delta_RA = −5049.0 mas delta_Dec = −6387.5 Epoch: 2022.271

## Establish these coordinates with astropy

In [1]:
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.time import Time

coord_ers = SkyCoord(ra=Angle("12:56:01.586", unit=u.hourangle), 
                     dec=Angle("−12:57:30.310", unit=u.deg),
                    obstime=Time(55743.067635, format='mjd'),
                    pm_ra_cosdec=-275.4*u.mas/u.yr, pm_dec=-198.4*u.mas/u.yr)

coord_APT = SkyCoord(ra=Angle("12:56:1.5389", unit=u.hourangle),
                     dec=Angle("−12:57:30.62", unit=u.deg),
                    obstime=Time(2015.5, format='jyear'),
                    pm_ra_cosdec=-284.2*u.mas/u.yr, pm_dec=-189.0*u.mas/u.yr)
                    
coord_A_simbad = SkyCoord(Angle("12:56:02.133704", unit=u.hourangle),
                          Angle("−12:57:21.9243", unit=u.deg),
                         obstime=Time(2000, format='jyear'),                    
                        pm_ra_cosdec=-272.5*u.mas/u.yr, pm_dec=-190.2*u.mas/u.yr)

# translate to the epoch when Dupuy measured the latest relative astrometry
coord_A_J2022 = coord_A_simbad.apply_space_motion(new_obstime=Time(2022.271, format='jyear'))
# make a copy and then apply the offset
coord_Dupuy = SkyCoord(ra=coord_A_J2022.ra+(-5049.0)*u.mas,
                         dec=coord_A_J2022.dec+(-6387.5)*u.mas,
                         obstime=Time(2022.271, format='jyear'),
                         pm_ra_cosdec=coord_A_J2022.pm_ra_cosdec,
                         pm_dec=coord_A_J2022.pm_dec)

/Users/nguyendat/opt/anaconda3/lib/python3.9/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 1 of "distance overridden (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [4]:
# set the nominal epoch to be close the observing day, say 2024-06-01

obsTime = Time('2024-06-01')

In [5]:
coord_ers_obs = coord_ers.apply_space_motion(new_obstime=obsTime)
coord_APT_obs = coord_APT.apply_space_motion(new_obstime=obsTime)
coord_Dupuy_obs = coord_Dupuy.apply_space_motion(new_obstime=obsTime)

In [6]:
print('Separation between ERS and APT: {0}'.format(coord_ers_obs.separation(coord_APT_obs).to(u.arcsec)))
print('Separation between ERS and Dupuy: {0}'.format(coord_ers_obs.separation(coord_Dupuy_obs).to(u.arcsec)))

Separation between ERS and APT: 0.6604052151940797 arcsec
Separation between ERS and Dupuy: 0.08242160002702464 arcsec


In [7]:
print('RA:', coord_Dupuy_obs.ra.hms)
print('Dec:', coord_Dupuy_obs.dec.dms)
print('Epoch:', coord_Dupuy_obs.obstime.jyear)

RA: hms_tuple(h=12.0, m=56.0, s=1.3420025257660484)
Dec: dms_tuple(d=-12.0, m=-57.0, s=-32.95523442092218)
Epoch: 2024.4147843942505


In [8]:
coord_A_J2022.position_angle(coord_Dupuy).degree-180-4.7

32.9076429833229

In [9]:
# propagate to the ERS proposal epoch to double check
coord_ers_backprop = coord_Dupuy.apply_space_motion(new_obstime=Time(2011.4950, format='jyear'))
print('RA:', coord_ers_backprop.ra.hms)
print('Dec:', coord_ers_backprop.dec.dms)
print('Epoch:', coord_ers_backprop.obstime.jyear)

RA: hms_tuple(h=12.0, m=56.0, s=1.582834674300102)
Dec: dms_tuple(d=-12.0, m=-57.0, s=-30.498034087444665)
Epoch: 2011.495


In [10]:
print('RA:', coord_ers.ra.hms)
print('Dec:', coord_ers.dec.dms)
print('Epoch:', coord_ers.obstime.jyear)

RA: hms_tuple(h=12.0, m=56.0, s=1.5860000000104435)
Dec: dms_tuple(d=-12.0, m=-57.0, s=-30.310000000001764)
Epoch: 2011.4950517043121


In [11]:
print('Separation between ERS and ERS_backprop: {0}'.format(coord_ers.separation(coord_ers_backprop).to(u.arcsec)))

Separation between ERS and ERS_backprop: 0.19364347897073875 arcsec
